In [8]:
import json
import pandas as pd
import re
import ast

# PlumX Metrics

In [9]:
f = open('plumx_metrics_data.json')
data_json = json.load(f)

In [10]:
len(data_json)

24515

In [11]:
data_flat = pd.DataFrame()
for item in data_json:
    item_flat =  pd.json_normalize(item)
    data_flat = pd.concat([data_flat, item_flat], axis=0, join='outer')

In [12]:
data_flat.shape

(24515, 20)

In [13]:
data_flat.to_csv("plumx_metrics_tabular_uncleaned.csv", index=False)

# Altmetrics

In [14]:
f = open('altmetric_data.json')
altmetrics_corpus = f.read()
altmetrics_corpus = altmetrics_corpus.replace('\\', '')
altmetrics_corpus = altmetrics_corpus.replace(',', ', ')
altmetrics_corpus = altmetrics_corpus.replace(':', ': ')
altmetrics_corpus = altmetrics_corpus[1:-1]

In [15]:
index = 0
item_start_index = None
item_end_index = None
total = len(altmetrics_corpus)
stack = 0
item_indices = list()
while(index < total):
    if(altmetrics_corpus[index]=='{'):
        if(stack == 0): # opening new item 
            item_start_index = index
            stack+=1
            index+=1 
        else: # nested data within item 
            stack+=1
            index+=1
    elif(altmetrics_corpus[index]=='}'):
        if(stack == 1): # closing item
            item_end_index = index+1
            item_indices.append((item_start_index, item_end_index))
            stack-=1
            index+=1
        else: # closing nested data within item
            stack-=1
            index+=1
    else:
        index+=1   

In [16]:
len(item_indices) # should be 8416

8416

In [17]:
data_flat = pd.DataFrame()
error_count = 0
for item in item_indices:
    text = altmetrics_corpus[item[0]:item[1]]
    try:
        item_dict = json.loads(text)
        item_flat =  pd.json_normalize(item_dict)
        data_flat = pd.concat([data_flat, item_flat], axis=0, join='outer')
    except:
        print("Unable to parse start index ", item[0], " end index ", item[1], " .")
        error_count +=1
print("Error count: ", error_count)

Unable to parse start index  121750  end index  126290  .
Unable to parse start index  184977  end index  188178  .
Unable to parse start index  294557  end index  297759  .
Unable to parse start index  324913  end index  329130  .
Unable to parse start index  744458  end index  747393  .
Unable to parse start index  1029325  end index  1033507  .
Unable to parse start index  1126755  end index  1131615  .
Unable to parse start index  1186233  end index  1190825  .
Unable to parse start index  1310325  end index  1313887  .
Unable to parse start index  1715926  end index  1719969  .
Unable to parse start index  1757374  end index  1761067  .
Unable to parse start index  1825899  end index  1829425  .
Unable to parse start index  1841094  end index  1844491  .
Unable to parse start index  2081659  end index  2090490  .
Unable to parse start index  2770268  end index  2775615  .
Unable to parse start index  3091781  end index  3094751  .
Unable to parse start index  3313544  end index  3

We will ignore the parsing errors for now.  They appear to be caused by the use of quotation marks in the abstract and 110 (1.3%) samples are affected. This will be addressed in a later update.

In [18]:
data_flat.shape

(8306, 100)

In [19]:
data_flat.to_csv("altmetrics_tabular_uncleaned.csv", index=False)